In [1]:
import numpy as np
import pandas as pd
import os
import math
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt

In [71]:
paths = []

for i in range(1, 16):
    root, file = os.path.split("../Dataset/SA01/F01_SA23_R01.csv")
    if i < 10:
        file = file[:2] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        paths.append(tmp)
        if i > 4:
            for j in range(2, 6):
                file_new = file[:-5] + str(j) + file[-4:]
                tmp = os.path.join(root, file_new)
                paths.append(tmp)
    if i >= 10:
        file = file[:1] + str(i) + file[3:]
        for j in range(1, 6):
            file = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file)
            paths.append(tmp)

# paths[4:14] = ['../Dataset/SA01/D05_SA01_R01_1.csv',
#                 '../Dataset/SA01/D05_SA01_R01_2.csv',
#                 '../Dataset/SA01/D05_SA01_R02_1.csv',
#                 '../Dataset/SA01/D05_SA01_R02_2.csv',
#                 '../Dataset/SA01/D05_SA01_R03_1.csv',
#                 '../Dataset/SA01/D05_SA01_R03_2.csv',
#                 '../Dataset/SA01/D05_SA01_R04_1.csv',
#                 '../Dataset/SA01/D05_SA01_R04_2.csv',
#                 '../Dataset/SA01/D05_SA01_R05_1.csv',
#                 '../Dataset/SA01/D05_SA01_R05_2.csv',
#                 '../Dataset/SA01/D06_SA01_R01_1.csv',
#                 '../Dataset/SA01/D06_SA01_R01_2.csv',
#                 '../Dataset/SA01/D06_SA01_R02_1.csv',
#                 '../Dataset/SA01/D06_SA01_R02_2.csv',
#                 '../Dataset/SA01/D06_SA01_R03_1.csv',
#                 '../Dataset/SA01/D06_SA01_R03_2.csv',
#                 '../Dataset/SA01/D06_SA01_R04_1.csv',
#                 '../Dataset/SA01/D06_SA01_R04_2.csv',
#                 '../Dataset/SA01/D06_SA01_R05_1.csv',
#                 '../Dataset/SA01/D06_SA01_R05_2.csv']

# paths[-15:-10] = ['../Dataset/SA01/D17_SA01_R01_1.csv',
#                  '../Dataset/SA01/D17_SA01_R01_2.csv',
#                  '../Dataset/SA01/D17_SA01_R02_1.csv',
#                  '../Dataset/SA01/D17_SA01_R02_2.csv',
#                  '../Dataset/SA01/D17_SA01_R03_1.csv',
#                  '../Dataset/SA01/D17_SA01_R03_2.csv',
#                  '../Dataset/SA01/D17_SA01_R04_1.csv',
#                  '../Dataset/SA01/D17_SA01_R04_2.csv',
#                  '../Dataset/SA01/D17_SA01_R05_1.csv',
#                 '../Dataset/SA01/D17_SA01_R05_2.csv']

SA23_F_paths = []

for path in paths:
    path = path.replace('SA01', 'SA23')
    SA23_F_paths.append(path)

SA23_F_paths

['../Dataset/SA23\\F01_SA23_R01.csv',
 '../Dataset/SA23\\F02_SA23_R01.csv',
 '../Dataset/SA23\\F03_SA23_R01.csv',
 '../Dataset/SA23\\F04_SA23_R01.csv',
 '../Dataset/SA23\\F05_SA23_R01.csv',
 '../Dataset/SA23\\F05_SA23_R02.csv',
 '../Dataset/SA23\\F05_SA23_R03.csv',
 '../Dataset/SA23\\F05_SA23_R04.csv',
 '../Dataset/SA23\\F05_SA23_R05.csv',
 '../Dataset/SA23\\F06_SA23_R01.csv',
 '../Dataset/SA23\\F06_SA23_R02.csv',
 '../Dataset/SA23\\F06_SA23_R03.csv',
 '../Dataset/SA23\\F06_SA23_R04.csv',
 '../Dataset/SA23\\F06_SA23_R05.csv',
 '../Dataset/SA23\\F07_SA23_R01.csv',
 '../Dataset/SA23\\F07_SA23_R02.csv',
 '../Dataset/SA23\\F07_SA23_R03.csv',
 '../Dataset/SA23\\F07_SA23_R04.csv',
 '../Dataset/SA23\\F07_SA23_R05.csv',
 '../Dataset/SA23\\F08_SA23_R01.csv',
 '../Dataset/SA23\\F08_SA23_R02.csv',
 '../Dataset/SA23\\F08_SA23_R03.csv',
 '../Dataset/SA23\\F08_SA23_R04.csv',
 '../Dataset/SA23\\F08_SA23_R05.csv',
 '../Dataset/SA23\\F09_SA23_R01.csv',
 '../Dataset/SA23\\F09_SA23_R02.csv',
 '../Dataset

In [49]:
def butter_lowpass(cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order=order)
    x = np.trunc(np.array(data, dtype=np.float64)).astype(np.int64)
    y = lfilter(b, a, x, axis=-1, zi=None)
    return y

# Setting standard filter requirements.
order = 4
fs = 200
cutoff = 5

b, a = butter_lowpass(cutoff, fs, order)

# Creating the data for filteration
T = 200         # value taken in seconds
n = int(T * fs) # indicates total samples
t = np.linspace(0, T, n, endpoint=False)

In [72]:
list = []
LPF_list = []
final_SA23_F = pd.DataFrame(list)

for path in SA23_F_paths:
    try:
        data = pd.read_csv(path, header=None)
        data = data.iloc[:, :3]
        rows, cols = data.shape

        sliced_df = data[0 : 160]
    
        # LPF 구하기

        col_1_lpf = butter_lowpass_filter(sliced_df[0].to_numpy(), cutoff, fs, order).astype('float64')
        col_2_lpf = butter_lowpass_filter(sliced_df[1].to_numpy(), cutoff, fs, order).astype('float64')
        col_3_lpf = butter_lowpass_filter(sliced_df[2].to_numpy(), cutoff, fs, order).astype('float64')
    
    
        col_1_std = np.sqrt(np.absolute(np.mean(sliced_df[0]) - sliced_df[0])**2 / len(sliced_df[0]))
        col_2_std = np.sqrt(np.absolute(np.mean(sliced_df[1]) - sliced_df[1])**2 / len(sliced_df[1]))
        col_3_std = np.sqrt(np.absolute(np.mean(sliced_df[2]) - sliced_df[2])**2 / len(sliced_df[2]))
    
        # 가속도 구하기
        # 표준편차 구하기

        col_1 = col_1_std*col_1_std
        col_2 = col_2_std*col_2_std
        col_3 = col_3_std*col_3_std
    
        C_nine = np.sqrt(col_1 + col_2 + col_3)
        list_C_nine = pd.DataFrame(C_nine).transpose().round()
        final_SA23_F = final_SA23_F.append(list_C_nine)
        
    except:
        pass

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

row, col = final_SA23_F.shape
target_col = row * [1]
final_SA23_F.loc[:,'target'] = target_col

final_SA23_F

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,target
0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,8.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,3.0,4.0,4.0,3.0,3.0,3.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,1
0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,...,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,3.0,3.0,3.0,4.0,4.0,5.0,5.0,4.0,4.0,4.0,3.0,3.0,3.0,4.0,4.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1
0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,1
0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,2.0,3.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,4.0,4.0,4.0,4.0,4.0,3.0,2.0,2.0,1
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,3.0,3.0,3.0,4.0,1
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,1
0,0.0,1.0,1.0,1.

In [7]:
A01 = final_SA01_F

In [10]:
A02 = final_SA02_F

In [13]:
A03 = final_SA03_F

In [16]:
A04 = final_SA04_F

In [19]:
A05 = final_SA05_F

In [22]:
A06 = final_SA06_F

In [25]:
A07 = final_SA07_F

In [28]:
A08 = final_SA08_F

In [31]:
A09 = final_SA09_F

In [34]:
A10 = final_SA10_F

In [37]:
A11 = final_SA11_F

In [40]:
A12 = final_SA12_F

In [43]:
A13 = final_SA13_F

In [46]:
A14 = final_SA14_F

In [49]:
A15 = final_SA15_F

In [52]:
A16 = final_SA16_F

In [55]:
A17 = final_SA17_F

In [58]:
A18 = final_SA18_F

In [61]:
A19 = final_SA19_F

In [64]:
A20 = final_SA20_F

In [67]:
A21 = final_SA21_F

In [70]:
A22 = final_SA22_F

In [73]:
A23 = final_SA23_F

In [4]:
E06 = final_SE06_F

In [74]:
FALL_all = pd.concat([A01, A02, A03, A04, A05, A06, A07, A08, A09, A10, A11, A12, A13, A14, A15, A16, A17, A18, A19, A20, A21, A22, A23, E06])

In [75]:
FALL_all.to_csv('[C9]FALL_all_noLPF.csv',index=False)

In [76]:
df_1 = pd.read_csv('C:/Users/kss29/Data_handling/study/[C9]ADL_all_noLPF.csv')

In [78]:
df_2 = pd.read_csv('C:/Users/kss29/Data_handling/study/[C9]FALL_all_noLPF.csv')

In [80]:
df = pd.concat([df_1,df_2])

In [82]:
df.shape

(4626, 161)

In [81]:
df.to_csv('C9-standard deviation_noLPF.csv',index=False)